In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sys
import itertools
import warnings
import random
from pandas.api.types import is_numeric_dtype
warnings.filterwarnings('ignore')

In [2]:
train_data_name=input("Enter the name of train data file [(ex) harris.dat] : ") # data name
test_data_name=input("Enter the name of test data file [(ex) harris.dat] : ") 
coding_fm=int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")) # data separator
separator_fm={coding_fm ==1 : " "}.get(True, ",")
res_pos=int(input("Enter the column position of the response variable : [from 1 to p] : "))
header=input("Does the data have column header? (y/n) : ")
if(header=="y") : trdata=pd.read_csv(train_data_name, sep=separator_fm) # loading data
else : trdata=pd.read_csv(train_data_name, sep=separator_fm, header=None) # loading data
if(header=="y") : tstdata=pd.read_csv(test_data_name, sep=separator_fm) # loading data
else : tstdata=pd.read_csv(test_data_name, sep=separator_fm, header=None) # loading data
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

data = pd.DataFrame(trdata)
tstdata = pd.DataFrame(tstdata)


Enter the name of train data file [(ex) harris.dat] : veh.dat
Enter the name of test data file [(ex) harris.dat] : vehtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p] : 19
Does the data have column header? (y/n) : n
Enter the output file name to export [(ex) result.txt] : result.txt


In [8]:
def bootstrap(dataset, B=101):
  boot = list()
  oob = list()
  while len(boot)<B:
    sample = list() 
    while len(sample) < len(dataset):
	    index = random.randrange(len(dataset))
	    sample.append(dataset.iloc[index]) 
    Z = pd.DataFrame(sample)
    oob.append(data[data.index.isin(Z[0].index) == False])
    boot.append(Z)
  return boot, oob

def most_frequent(data):
    return max(data, key=data.count)

In [9]:
def baggingLDA(data,  n_bootstrapping=101):
  clf = LinearDiscriminantAnalysis()
  bag = {}
  b= bootstrap(data, B = n_bootstrapping)
  ooberr = []
  lab = []
  p =pd.DataFrame(index = range(n_bootstrapping), columns = range(len(data.columns)-1))
  m = np.round((len(data.columns)-1))
  for i in range(n_bootstrapping):
    boot = b[0]
    oob = b[1]
    label = random.sample(list(boot[i].drop([data.columns[res_pos-1]], axis=1 ).columns.values),int(m))
    lab.append(label)
    newX = boot[i].drop([data.columns[res_pos-1]], axis=1 ).loc[:,label]
    bag[f'LDA{i}'] = clf.fit(newX,boot[i].iloc[:, res_pos-1 ].values)
    ooby = oob[i].iloc[:, res_pos-1 ].values
    oobpredy = clf.predict(newX )
    ooberr.append(len(ooby[(ooby != oobpredy) ==True ]) / len(ooby))

    for j in label:
      poob = oob[i].drop([data.columns[res_pos-1]], axis=1 ).iloc[:,label]
      poob.reset_index(drop = True, inplace = True)
      H = pd.Series(data = poob.loc[:,j].sample(n=len(oob[i]),replace = False))
      H.reset_index(drop = True,inplace = True)
      poob[j] = H
      poobpredy = clf.predict(poob)
      p[j][i] = (len(ooby[(ooby != poobpredy) ==True ]) / len(ooby))       


   #i번째 변수의 중요도 계산
  f=[]
  for j in list(data.drop([data.columns[res_pos-1]], axis=1 ).columns.values):
    ppp = []
    icollect = []
    for i in range(n_bootstrapping):
      if j in lab[i]:
        icollect.append(i)
        ppp.append(p[j][i])
    eee=[]
    for k in icollect:
      eee.append(ooberr[k])
    eee= np.array(eee)
    ppp = np.array(ppp)
    d = ppp - eee
    f.append((np.sum(d)/n_bootstrapping)/(np.std(d)*np.sqrt(len(d)-1)/np.sqrt(n_bootstrapping-1)))
  return bag, f ,lab

In [10]:
def rfLDA(data,  n_bootstrapping=101):
  clf = LinearDiscriminantAnalysis()
  bag = {}
  b= bootstrap(data, B = n_bootstrapping)
  ooberr = []
  lab = []
  p =pd.DataFrame(index = range(n_bootstrapping), columns = range(len(data.columns)-1))
  m = np.round((len(data.columns)-1)/2)
  for i in range(n_bootstrapping):
    boot = b[0]
    oob = b[1]
    label = random.sample(list(boot[i].drop([data.columns[res_pos-1]], axis=1 ).columns.values),int(m))
    lab.append(label)
    newX = boot[i].drop([data.columns[res_pos-1]], axis=1 ).loc[:,label]
    bag[f'LDA{i}'] = clf.fit(newX,boot[i].iloc[:, res_pos-1 ].values)
    ooby = oob[i].iloc[:, res_pos-1 ].values
    oobpredy = clf.predict(newX )
    ooberr.append(len(ooby[(ooby != oobpredy) ==True ]) / len(ooby))

    for j in label:
      poob = oob[i].drop([data.columns[res_pos-1]], axis=1 ).iloc[:,label]
      poob.reset_index(drop = True, inplace = True)
      H = pd.Series(data = poob.loc[:,j].sample(n=len(oob[i]),replace = False))
      H.reset_index(drop = True,inplace = True)
      poob[j] = H
      poobpredy = clf.predict(poob)
      p[j][i] = (len(ooby[(ooby != poobpredy) ==True ]) / len(ooby))       


   #i번째 변수의 중요도 계산
  f=[]
  for j in list(data.drop([data.columns[res_pos-1]], axis=1 ).columns.values):
    ppp = []
    icollect = []
    for i in range(n_bootstrapping):
      if j in lab[i]:
        icollect.append(i)
        ppp.append(p[j][i])
    eee=[]
    for k in icollect:
      eee.append(ooberr[k])
    eee= np.array(eee)
    ppp = np.array(ppp)
    d = ppp - eee
    f.append((np.sum(d)/n_bootstrapping)/(np.std(d)*np.sqrt(len(d)-1)/np.sqrt(n_bootstrapping-1)))
  return bag, f ,lab

In [11]:
def result(tstdata, method,n_bootstrapping=101):
  print('Variable Importance: ')
  for i in range(len(data.columns)-1):
    print(f'   X{i+1}:', Z[1][i])

  predy = []
  for i in range(n_bootstrapping):
    X = tstdata.loc[:,Z[2][i]]
    pred = Z[0][f'LDA{i}'].predict(X)
    predy.append(pred)
  
  newy=[]
  for j in range(len(tstdata)):
    r = []
    for i in range(n_bootstrapping):
      r.append(predy[i][j])
    newy.append(most_frequent(r))


  print('    ')
  print( 'Confusion Matrix( LDA - ', method, ')')
  print('----------------------------------------------------')
  confusion_tst = confusion_matrix(tstdata.iloc[:,res_pos-1], newy)

  accu_tst = 0
  for i in range(len(np.unique(data.iloc[:,res_pos-1]))):
    accu_tst = accu_tst + confusion_tst[i][i]
  accuracy_tst = accu_tst / len(tstdata)

  print('           predicted class \n Actual 1 ' ,confusion_tst[0], '\n class  2 ', confusion_tst[1])
  for i in range(2, len(np.unique(data.iloc[:,res_pos-1]))) :
    print(f'        {i+1} ', confusion_tst[i])
  print('model summary')
  print('----------------------------------')
  print('Overall accuracy = ' ,accuracy_tst)



In [22]:
sys.stdout = open(out_name,'w')
Z = baggingLDA(data)
result(tstdata, method = 'Bagging')
print('    ')
print('    ')
print('    ')

In [25]:
Z = rfLDA(data)
result(tstdata, method = 'Random Feature')